# Importing Libraries

In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# Load environment & Check the key

In [2]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
openai = OpenAI()

if api_key:
    print("API key found and looks good so far!")
else:
    print("API key not found. Please check your .env file.")

API key found and looks good so far!


# Define Class to represent Webpage

In [3]:
class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        self.title = soup.title.string if soup.title else "there is no title"
        for irrelevent in soup.body(["script", "style", "img", "input"]):
            irrelevent.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [4]:
commerce = Website(
    "https://learn.microsoft.com/en-us/dynamics365/commerce/channel-setup-retail"
)
print(commerce.title)

Set up a retail  channel - Commerce | Dynamics 365 | Microsoft Learn


In [5]:
system_prompt = """
You are an assistant tasked with analyzing a website's content and offering a brief summary. 
Avoid including text that seems related to navigation. Reply in markdown format.
"""

In [6]:
def user_prompt_for(Website):
    user_prompt = f"the Website title is {Website.title}"
    user_prompt += """
    The contents of this website is as follows; \
    please provide a short summary of this website in markdown. \
    If it includes news or announcements, then summarize these too.\n\n
    """
    user_prompt += Website.text
    return user_prompt

The API from OpenAI expects to receive messages in a particular structure.  
Many of the other APIs share this structure:  

[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]


In [7]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)},
    ]

In [8]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model="gpt-4o-mini", messages=messages_for(website)
    )
    return response.choices[0].message.content

In [9]:
summarize("https://learn.microsoft.com/en-us/dynamics365/commerce/channel-setup-retail")

'# Summary of "Set up a retail channel - Commerce | Dynamics 365 | Microsoft Learn"\n\nThis webpage provides a comprehensive guide on creating and configuring a retail channel using Microsoft Dynamics 365 Commerce. It emphasizes the support for multiple retail channels including online stores, call centers, and brick-and-mortar locations, each customizable with unique settings such as payment methods, price groups, and staff assignments. \n\n## Key Content Highlights:\n\n- **Retail Channel Creation Steps**: Instructions on setting up a new retail channel, including details on naming, identifying store numbers and legal entities, selecting time zones, and setting up various profiles.\n\n- **Payment Methods**: Detailed processes to configure payment methods for each channel, including cash declarations and modes of delivery.\n\n- **Additional Configuration**: Guidance on setting up income/expense accounts, fulfillment groups, and ensuring unique transaction IDs using a new format to avoi

# A function to display this nicely in the Jupyter output, using markdown

In [10]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [11]:
display_summary(
    "https://learn.microsoft.com/en-us/dynamics365/commerce/channel-setup-retail"
)

# Summary: Set Up a Retail Channel - Dynamics 365 Commerce

This article from Microsoft Learn provides a comprehensive guide on how to create and configure a new retail channel in Microsoft Dynamics 365 Commerce. It outlines the different types of retail channels supported, such as online stores, call centers, and brick-and-mortar locations. 

## Key Points:

- **Channel Setup Prerequisites**: Before creating a retail channel, certain prerequisites must be fulfilled.
- **Creation Procedure**: The guide details step-by-step instructions for setting up a new retail channel, including necessary configurations such as store name, store number, legal entity, warehouse, timezone, sales tax group, and default customer settings.
- **Payment Methods**: Instructions are provided for setting up various payment methods, cash declarations, modes of delivery, income/expense accounts, sections, fulfillment group assignments, and safes.
- **Transaction ID Format**: A new 13-digit transaction ID format is introduced to prevent duplicate transaction IDs, which is particularly beneficial in both online and offline sales modes.
- **Store Locations**: It covers how to set up store location data that can be used in POS and e-commerce scenarios.

## Announcement:
- The new transaction ID format feature is enabled by default starting with Commerce version 10.0.41, ensuring uniqueness in transaction IDs. 

This guide serves as a detailed reference for retail professionals looking to set up and manage their retail channels effectively within the Dynamics 365 Commerce framework.